# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import json
import os

from pathlib import Path

In [12]:
from dotenv import load_dotenv
load_dotenv()
google_api_key = os.getenv("google_api_key")
type(google_api_key)

NoneType

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_df = pd.read_csv('clean_city_data.csv')
city_df

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,tiznit,29.5833,-9.5000,79.95,19,1,0.51,MA,1634462485
1,sokolo,14.7328,-6.1219,90.27,28,93,2.98,ML,1634462486
2,maldonado,-34.6667,-54.9167,47.23,76,0,4.72,UY,1634462486
3,kalabo,-14.9700,22.6814,94.37,25,5,7.40,ZM,1634462487
4,ushuaia,-54.8000,-68.3000,33.46,86,75,12.66,AR,1634462250
...,...,...,...,...,...,...,...,...,...
551,labuhan,-6.8844,112.2051,87.37,64,75,13.91,ID,1634462734
552,mareeba,-17.0000,145.4333,76.48,83,62,6.53,AU,1634462734
553,sale,34.0389,-6.8166,71.62,94,100,0.00,MA,1634462533
554,kenai,60.5544,-151.2583,34.14,92,90,8.05,US,1634462735


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Store latitude and longitude in locations
locations = city_df[["Latitude", "Longitude"]]
locations

,Latitude,Longitude
0,29.5833,-9.5000
1,14.7328,-6.1219
2,-34.6667,-54.9167
3,-14.9700,22.6814
4,-54.8000,-68.3000
...,...,...
551,-6.8844,112.2051
552,-17.0000,145.4333
553,34.0389,-6.8166
554,60.5544,-151.2583


In [20]:
# Would fill the NaN values but we already checked it last one, ahh its all coming together, ohh and convert to float
rating = city_df["Humidity"].astype(float)

In [21]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=rating,
                                    dissipating= False,
                                    max_intensity=30,
                                    point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

NameError: name 'Fig' is not defined

In [21]:
city_df[["Max Temp"]].describe()

,Max Temp
count,556.000000
mean,62.283597
std,20.643020
min,-12.280000
25%,48.600000
50%,64.445000
75%,79.955000
max,98.370000


In [4]:
city_df_filtered = city_df[city_df['Max Temp'] >= 85]
city_df_filtered

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,sokolo,14.7328,-6.1219,90.27,28,93,2.98,ML,1634462486
3,kalabo,-14.9700,22.6814,94.37,25,5,7.40,ZM,1634462487
7,san policarpo,13.3034,-4.8956,88.30,39,27,6.49,ML,1634462302
12,raga,8.4596,25.6780,87.51,54,31,2.95,SS,1634462491
25,banda aceh,25.4833,80.3333,95.23,46,12,12.19,IN,1634462495
...,...,...,...,...,...,...,...,...,...
534,dhola,21.8833,71.7833,91.13,30,36,6.11,IN,1634462726
543,solwezi,-12.1688,26.3894,93.90,22,37,3.58,ZM,1634462731
545,buraydah,26.3260,43.9750,97.25,9,0,4.43,SA,1634462732
549,mitsamiouli,-11.3847,43.2844,86.29,54,20,28.77,KM,1634462733


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# not a lot of conditions given on this, but lets say that nobody wants to go on vacation in places greater that 85 degrees
city_df.drop(city_df[city_df['Max Temp'] >= 85].index, inplace = True)
city_df

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,tiznit,29.5833,-9.5000,79.95,19,1,0.51,MA,1634462485
2,maldonado,-34.6667,-54.9167,47.23,76,0,4.72,UY,1634462486
4,ushuaia,-54.8000,-68.3000,33.46,86,75,12.66,AR,1634462250
5,touros,-5.1989,-35.4608,74.30,83,40,13.89,BR,1634462487
6,rikitea,-23.1203,-134.9692,75.06,75,7,14.20,PF,1634462386
...,...,...,...,...,...,...,...,...,...
550,lata,40.1629,-8.3327,68.92,95,100,2.44,PT,1634462734
552,mareeba,-17.0000,145.4333,76.48,83,62,6.53,AU,1634462734
553,sale,34.0389,-6.8166,71.62,94,100,0.00,MA,1634462533
554,kenai,60.5544,-151.2583,34.14,92,90,8.05,US,1634462735


In [6]:
# and while were at it nobody probably wants to go to places below freezing, if it was skiing season maybe, but its October...
# its skiing season nowhere, its only 46 locations
city_df.drop(city_df[city_df['Max Temp'] <= 32].index, inplace = True)
city_df

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,tiznit,29.5833,-9.5000,79.95,19,1,0.51,MA,1634462485
2,maldonado,-34.6667,-54.9167,47.23,76,0,4.72,UY,1634462486
4,ushuaia,-54.8000,-68.3000,33.46,86,75,12.66,AR,1634462250
5,touros,-5.1989,-35.4608,74.30,83,40,13.89,BR,1634462487
6,rikitea,-23.1203,-134.9692,75.06,75,7,14.20,PF,1634462386
...,...,...,...,...,...,...,...,...,...
550,lata,40.1629,-8.3327,68.92,95,100,2.44,PT,1634462734
552,mareeba,-17.0000,145.4333,76.48,83,62,6.53,AU,1634462734
553,sale,34.0389,-6.8166,71.62,94,100,0.00,MA,1634462533
554,kenai,60.5544,-151.2583,34.14,92,90,8.05,US,1634462735


In [7]:
# if its humid and cloudy then that means its likely rianing, and nobody wants to go to a place that is grey and overcast
#thats not an ideal vacation destination
#city_df.drop(city_df[["Humidity", "Cloudiness"] <= 90].index, inplace = True)
humid_df = city_df[city_df["Humidity"] <= 90]
humid_df

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,tiznit,29.5833,-9.5000,79.95,19,1,0.51,MA,1634462485
2,maldonado,-34.6667,-54.9167,47.23,76,0,4.72,UY,1634462486
4,ushuaia,-54.8000,-68.3000,33.46,86,75,12.66,AR,1634462250
5,touros,-5.1989,-35.4608,74.30,83,40,13.89,BR,1634462487
6,rikitea,-23.1203,-134.9692,75.06,75,7,14.20,PF,1634462386
...,...,...,...,...,...,...,...,...,...
544,chaykovskiy,56.7686,54.1148,52.81,49,77,12.17,RU,1634462731
546,ulladulla,-35.3500,150.4667,58.10,72,68,6.91,AU,1634462726
547,nizhniy odes,45.3926,39.2364,68.99,52,100,5.57,RU,1634462732
548,kirakira,-10.4544,161.9205,80.10,81,39,3.20,SB,1634462733


In [8]:
#nobody wants to go to place that is grey and overcast, thats not an ideal vacation destination
# theres 144 of them
hotel_df = humid_df[humid_df['Cloudiness'] <= 90]
hotel_df

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,tiznit,29.5833,-9.5000,79.95,19,1,0.51,MA,1634462485
2,maldonado,-34.6667,-54.9167,47.23,76,0,4.72,UY,1634462486
4,ushuaia,-54.8000,-68.3000,33.46,86,75,12.66,AR,1634462250
5,touros,-5.1989,-35.4608,74.30,83,40,13.89,BR,1634462487
6,rikitea,-23.1203,-134.9692,75.06,75,7,14.20,PF,1634462386
...,...,...,...,...,...,...,...,...,...
542,mananara,-16.1667,49.7667,78.26,66,44,8.77,MG,1634462730
544,chaykovskiy,56.7686,54.1148,52.81,49,77,12.17,RU,1634462731
546,ulladulla,-35.3500,150.4667,58.10,72,68,6.91,AU,1634462726
548,kirakira,-10.4544,161.9205,80.10,81,39,3.20,SB,1634462733


In [9]:
hotel_df["Hotel Name"] = ""

C:\Users\mdebo\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
hotel_df


,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,tiznit,29.5833,-9.5000,79.95,19,1,0.51,MA,1634462485,
2,maldonado,-34.6667,-54.9167,47.23,76,0,4.72,UY,1634462486,
4,ushuaia,-54.8000,-68.3000,33.46,86,75,12.66,AR,1634462250,
5,touros,-5.1989,-35.4608,74.30,83,40,13.89,BR,1634462487,
6,rikitea,-23.1203,-134.9692,75.06,75,7,14.20,PF,1634462386,
...,...,...,...,...,...,...,...,...,...,...
542,mananara,-16.1667,49.7667,78.26,66,44,8.77,MG,1634462730,
544,chaykovskiy,56.7686,54.1148,52.81,49,77,12.17,RU,1634462731,
546,ulladulla,-35.3500,150.4667,58.10,72,68,6.91,AU,1634462726,
548,kirakira,-10.4544,161.9205,80.10,81,39,3.20,SB,1634462733,


In [29]:
params = {
        "radius": 50000,
#        "rankby": "distance",
        "type" : "lodging",
        "query": "hotel",
        "key": google_api_key
    }
base_url = "https://maps.googleapis.com/maps/api/place/textsearch/json"

In [13]:
base_url = "https://maps.googleapis.com/maps/api/place/textsearch/json?"

params = {
        "radius": 50000,
        "type" : "lodging",
        "query": "hotel",
        "key": google_api_key
    }

for index, row in hotel_df.iterrows():
    lat = row["Latitude"]
    lng = row["Longitude"]
    city_name = row["City"]
    
    params["location"] = f"{lat},{lng}"
    
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params)
    hotel_name = response.json()
    
    #Save the hotel name to the dataframe
    try:
        print(f"Closest hotel is {hotel_name}.")
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]['name']
    except (KeyError, IndexError):
        print("Could not find result")
        print("-------------------------")

        
        print("--------------------Ending Search")

Retrieving Results for Index 0: tiznit.
Closest hotel is {'error_message': 'You must use an API key to authenticate each request to Google Maps Platform APIs. For additional information, please refer to http://g.co/dev/maps-no-account', 'html_attributions': [], 'results': [], 'status': 'REQUEST_DENIED'}.
Could not find result
-------------------------
--------------------Ending Search
Retrieving Results for Index 2: maldonado.
Closest hotel is {'error_message': 'You must use an API key to authenticate each request to Google Maps Platform APIs. For additional information, please refer to http://g.co/dev/maps-no-account', 'html_attributions': [], 'results': [], 'status': 'REQUEST_DENIED'}.
Could not find result
-------------------------
--------------------Ending Search
Retrieving Results for Index 4: ushuaia.
Closest hotel is {'error_message': 'You must use an API key to authenticate each request to Google Maps Platform APIs. For additional information, please refer to http://g.co/dev/m

KeyboardInterrupt: 

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure


In [ ]:
from os import path
if not path.exists("Hotel_Output.csv"):
    print("------------------------")

    # params dictionary to update each iteration
    params = {
        "radius": 50000,
#        "rankby": "distance",
        "type" : "lodging",
        "query": "hotel",
        "key": google_api_key
        }
    base_url = "https://maps.googleapis.com/maps/api/place/textsearch/json"
    
    # use the lat/lng we have to identify hotels
    for index, row in hotel_df.iterrows():
        # get lat, lng from df
        lat = row["Latitude"]
        lng = row["Longitude"]
#        if row["Hotel Name"]=="":
            
        # change location each iteration while leaving original params in place
        params["location"] = f"{lat},{lng}"
            
        hotel_name = requests.get(base_url, params=params)
        print(f"Retrieving Results for Index {index}: {row['City']}.")
        # extract results
        
            
        # convert to json
        hotel_name = hotel_name.json()    
        print(json.dumps(hotel_name, indent=4, sort_keys=True))
            
            # since some data may be missing we incorporate a try-except to skip any that are
        try:
            print(f"Closest hotel is {hotel_name}.")
            hotel_df.loc[index, "Hotel Name"] = hotel_name[results][0]["name"]
                
        except (KeyError, IndexError):
            print("Missing field/result.... skipping.")
            
# save data to csv
hotel_df.to_csv("Hotel_Output.csv", index=False)        